In [ ]:
!pip install langchain_community langchain_groq youtube_transcript_api faiss-cpu

In [2]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
import textwrap

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [4]:
def create_db_from_youtube_video_url(video_url):
  loader = YoutubeLoader.from_youtube_url(video_url)
  transcript = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 100)
  docs = text_splitter.split_documents(transcript)
  db = FAISS.from_documents(docs, embeddings)
  return db

In [5]:
def get_response_from_query(db, query, k = 4):
  docs = db.similarity_search(query, k = k)
  docs_page_content = " ".join([d.page_content for d in docs])

  chat = ChatGroq(api_key = "your-api-key", model = "llama-3.3-70b-versatile", temperature=0)

  template = """You are a helpful assistant that that can answer questions about youtube videos
        based on the video's transcript: {docs}

        Only use the factual information from the transcript to answer the question.

        If you feel like you don't have enough information to answer the question, say "I don't know"."""

  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template = "Answer the following question: {question}"

  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

  chat_prompt = ChatPromptTemplate.from_messages(
      [system_message_prompt, human_message_prompt]
  )

  chain = LLMChain(llm = chat, prompt = chat_prompt)

  response = chain.run(question = query, docs = docs_page_content)

  response = response.replace("\n", "")

  return response, docs

In [6]:
video_url = "https://youtu.be/fNBTtCkgg54?si=UU2_2GntrrTTLi7Z"
db = create_db_from_youtube_video_url(video_url)
query = "what is this video about?"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response, width = 70))

<ipython-input-5-ff160b2447e2>:23: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = chat, prompt = chat_prompt)
<ipython-input-5-ff160b2447e2>:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(question = query, docs = docs_page_content)


This video appears to be a crash course for interview preparation,
specifically covering topics such as constructors, destructors, and
try-catch blocks, with the goal of providing viewers with knowledge
and examples to help them prepare for interviews.


In [7]:
print(docs)

[Document(id='51e4d755-5aa7-4836-9bc6-4dd5266e291e', metadata={'source': 'fNBTtCkgg54'}, page_content="this video okay which will totally motivate me whoever is watching it is a very easy every person need to put only one like and you done then after that please comment down all the homeworks whatever I give in the video and what you feel about this crash course whether it is you feel good or not whether you love it or not and what is the quality do let me know I tried my best to create the best hopes crash course for the interview preparation so we'll meet you with a new series or with a new crash course very soon please do the task read it learn it and if you able to crack any interview or you found any interview questions do comment down below we love to read it so we'll meet you with a new video soon"), Document(id='50c2c2d7-50ac-46b9-a799-a21b3a8d0896', metadata={'source': 'fNBTtCkgg54'}, page_content="Robo that we all know or the ra is made by inheriting two of the classes togeth